# Testing branch refactor-offline

In [1]:
import km3io as ki

In [2]:
file = "~/km3net/orca4data/mc_data/data/reco/datav6.0test.jchain.aanet.00005971.root"

In [28]:
r1 = ki.OfflineReader(file)
r = ki.OfflineReader("/home/zineb/km3net/km3net/km3io/examples/samples/aanet_v2.0.0.root")
r3 = ki.OfflineReader("/home/zineb/km3net/km3net/km3io/examples/samples/numucc.root")

# explore events

In [29]:
len(r.events)

10

In [30]:
r.header.DAQ

/home/zineb/km3net/km3net/km3io/km3io/offline.py:126: UserWarning: Your file header has an unsupported format
  warnings.warn("Your file header has an unsupported format")


AttributeError: 'NoneType' object has no attribute 'DAQ'

In [31]:
r.events._keymap.keys()

dict_keys(['mc_run_id', 'flags', 'det_id', 'w', 'trigger_counter', 'run_id', 'overlays', 'w3list', 'frame_index', 'mc_id', 'comment', 'w2list', 'trigger_mask', 'id', 'index', 'mc_t', 't_sec', 't_ns', 'n_hits', 'n_mc_hits', 'n_tracks', 'n_mc_tracks'])

In [32]:
for key in r.events._keymap.keys():
    print(key, getattr(r.events, key))

mc_run_id [0 0 0 ... 0 0 0]
flags [0 0 0 ... 0 0 0]
det_id [44 44 44 ... 44 44 44]
w [[] [] [] ... [] [] []]
trigger_counter [0 0 1 ... 3 4 0]
run_id [5971 5971 5971 ... 5971 5971 5971]
overlays [60 19 95 ... 6 51 22]
w3list [[] [] [] ... [] [] []]
frame_index [182 183 202 ... 185 185 204]
mc_id [0 0 0 ... 0 0 0]
comment [b'' b'' b'' ... b'' b'' b'']
w2list [[] [] [] ... [] [] []]
trigger_mask [22 22 22 ... 22 22 22]
id [1 2 3 ... 8 9 10]
index [0 0 0 ... 0 0 0]
mc_t [0.0 0.0 0.0 ... 0.0 0.0 0.0]
t_sec [1567036818 1567036818 1567036820 ... 1567036818 1567036818 1567036820]
t_ns [200000000 300000000 200000000 ... 500000000 500000000 400000000]
n_hits [176 125 318 ... 84 255 105]
n_mc_hits [0 0 0 ... 0 0 0]
n_tracks [56 55 56 ... 56 54 56]
n_mc_tracks [0 0 0 ... 0 0 0]


In [33]:
r.events.det_id

<ChunkedArray [44 44 44 ... 44 44 44] at 0x7f7489db0110>

In [34]:
r.events.mc_run_id

<ChunkedArray [0 0 0 ... 0 0 0] at 0x7f748981ef90>

In [35]:
r.events.trigger_mask

<ChunkedArray [22 22 22 ... 22 22 22] at 0x7f74897b9110>

In [36]:
r.events.overlays

<ChunkedArray [60 19 95 ... 6 51 22] at 0x7f74897b9750>

In [37]:
r.events.w3list

<ChunkedArray [[] [] [] ... [] [] []] at 0x7f74897c32d0>

In [38]:
r.events.flags

<ChunkedArray [0 0 0 ... 0 0 0] at 0x7f748984ce10>

In [39]:
r.events.w

<ChunkedArray [[] [] [] ... [] [] []] at 0x7f748981e090>

In [40]:
r.events["overlays"]

<ChunkedArray [60 19 95 ... 6 51 22] at 0x7f74897c3790>

## test events slicing

In [41]:
r.events[2:8:2].n_hits

<ChunkedArray [318 83 71] at 0x7f74897c3b90>

In [42]:
r.events.n_hits[2:8:2]

<ChunkedArray [318 83 71] at 0x7f74897c7810>

In [43]:
s = slice(2, 8, 2)
s

slice(2, 8, 2)

In [44]:
r.events[s].n_hits

<ChunkedArray [318 83 71] at 0x7f74897c8fd0>

In [45]:
str(r.events)

'Number of elements: 10'

In [46]:
repr(r.events)

'<Branch[events]: 10 elements>'

# BUG IN EVENTS SLICING!

In [47]:
# this slicing sctructure is not even valid to make custom slices and yet it returns the value of 
# the second index "8"
r.events[2, 8, 2].n_hits

255

In [48]:
r.events[8].n_hits

255

In [49]:
r.events[2:8:2, 5][0].n_hits

176

In [50]:
# the last index is always picked up even if the rest of slicing indices don't make sense
r.events[2:8:2, 5][0][2].n_hits

318

In [51]:
# same here!
r.events[5, 2:8:2, 0][0].n_hits

176

In [52]:
# a similar index error should appear when "non sense" slicing is used
r.events[0][2].n_hits

318

In [53]:
r.events.n_hits[0][2]

IndexError: invalid index to scalar variable.

In [54]:
# this shouldn't be allowed
r.events[0:3, 4].n_hits

83

In [55]:
# this doesn't make sense, since we selected the first 3 elements, there shouldn't be a forth n_hits available!
r.events[0:3][4].n_hits

83

In [56]:
# we need similar IndexError ...
r.events[0:3].n_hits[4]

IndexError: index 4 out of bounds for length 3

In [57]:
# we need similar IndexError
r.events.n_hits[0:3, 2]

IndexError: too many indices for array

In [137]:
# slicing with a list: this returns only one element while r.events[[1, 2, 5]] returns 3
r.events[1, 2, 5]

<Branch[events]: 1 element>

In [138]:
r.events[[1, 2, 5]]

<Branch[events]: 3 elements>

# Explore hits

In [58]:
r.events.hits

<Branch[hits]: 10 elements>

In [59]:
r.events.hits._keymap.keys()

dict_keys(['pure_t', 'id', 'pos_y', 'tdc', 'dir_x', 'trig', 'pos_z', 'tot', 'channel_id', 'dir_y', 'dir_z', 't', 'dom_id', 'pos_x', 'pattern_flags', 'type'])

In [60]:
for key in r.events.hits._keymap.keys():
    print(key, getattr(r.events.hits, key), "\n")

pure_t [[0.0 0.0 0.0 ... 0.0 0.0 0.0] [0.0 0.0 0.0 ... 0.0 0.0 0.0] [0.0 0.0 0.0 ... 0.0 0.0 0.0] ... [0.0 0.0 0.0 ... 0.0 0.0 0.0] [0.0 0.0 0.0 ... 0.0 0.0 0.0] [0.0 0.0 0.0 ... 0.0 0.0 0.0]] 

id [[0 0 0 ... 0 0 0] [0 0 0 ... 0 0 0] [0 0 0 ... 0 0 0] ... [0 0 0 ... 0 0 0] [0 0 0 ... 0 0 0] [0 0 0 ... 0 0 0]] 

pos_y [[0.0 0.0 0.0 ... 0.0 0.0 0.0] [0.0 0.0 0.0 ... 0.0 0.0 0.0] [0.0 0.0 0.0 ... 0.0 0.0 0.0] ... [0.0 0.0 0.0 ... 0.0 0.0 0.0] [0.0 0.0 0.0 ... 0.0 0.0 0.0] [0.0 0.0 0.0 ... 0.0 0.0 0.0]] 

tdc [[0 0 0 ... 0 0 0] [0 0 0 ... 0 0 0] [0 0 0 ... 0 0 0] ... [0 0 0 ... 0 0 0] [0 0 0 ... 0 0 0] [0 0 0 ... 0 0 0]] 

dir_x [[0.0 0.0 0.0 ... 0.0 0.0 0.0] [0.0 0.0 0.0 ... 0.0 0.0 0.0] [0.0 0.0 0.0 ... 0.0 0.0 0.0] ... [0.0 0.0 0.0 ... 0.0 0.0 0.0] [0.0 0.0 0.0 ... 0.0 0.0 0.0] [0.0 0.0 0.0 ... 0.0 0.0 0.0]] 

trig [[1 1 0 ... 1 1 0] [0 0 0 ... 0 0 0] [1 0 0 ... 1 1 0] ... [0 0 0 ... 0 0 0] [0 1 1 ... 1 1 0] [1 1 1 ... 1 1 0]] 

pos_z [[0.0 0.0 0.0 ... 0.0 0.0 0.0] [0.0 0.0 0.0 ... 0.0

## test hits slicing

In [61]:
r.events.hits[1:3].dom_id

<ChunkedArray [[806451572 806451572 806451572 ... 809524432 809526097 809544061] [806451572 806451572 806451572 ... 809544061 809544061 809544061]] at 0x7f74897c3550>

In [62]:
r.events.hits.dom_id[1:3]

<ChunkedArray [[806451572 806451572 806451572 ... 809524432 809526097 809544061] [806451572 806451572 806451572 ... 809544061 809544061 809544061]] at 0x7f74897b9190>

In [63]:
r.events.hits.dom_id[0][1:3]

array([806451572, 806451572], dtype=int32)

In [64]:
r.events.hits[0].dom_id[1:3]

array([806451572, 806451572], dtype=int32)

In [65]:
r.events.hits[2:8:2].dom_id[0][0:5]

array([806451572, 806451572, 806451572, 806455814, 806455814], dtype=int32)

In [66]:
r.events.hits[2].dom_id[0:5]

array([806451572, 806451572, 806451572, 806455814, 806455814], dtype=int32)

## BUGS IN HITS SLICING

In [67]:
# the following 4 slicing are not equivalent!
r.events.hits[1:3][0].dom_id[0:7]

array([806451572, 806451572, 806451572, 806451572, 806455814, 806455814,
       806455814], dtype=int32)

In [68]:
r.events[1:3][0].hits.dom_id[0:7]

array([806451572, 806451572, 806451572, 806451572, 806455814, 806455814,
       806455814], dtype=int32)

In [69]:
r.events.hits.dom_id[1:3][0][0:7]

array([806451572, 806451572, 806451572, 806451572, 806451572, 806455814,
       806487226], dtype=int32)

In [70]:
r.events[1].hits.dom_id[0:7]

array([806451572, 806451572, 806451572, 806451572, 806451572, 806455814,
       806487226], dtype=int32)

In [71]:
# same slicing issues observed in events are in hits
r.events.hits[2, 8, 2]

<Branch[hits]: 1 element>

In [72]:
r.events.hits[2:8:2]

<Branch[hits]: 3 elements>

In [79]:
# when multiples indexes are given, the last one is the one picked no matter how many indexes you provide
r.events.hits[1:3][0][2].dom_id[0:5]

array([806451572, 806451572, 806451572, 806455814, 806455814], dtype=int32)

In [80]:
r.events.hits[1:3][3][0][10][2].dom_id[0:5]

array([806451572, 806451572, 806451572, 806455814, 806455814], dtype=int32)

In [81]:
r.events.hits[2].dom_id[0:5]

array([806451572, 806451572, 806451572, 806455814, 806455814], dtype=int32)

In [82]:
r.events.hits[2].dom_id[0:5]

array([806451572, 806451572, 806451572, 806455814, 806455814], dtype=int32)

In [83]:
r.events.hits.dom_id[2][0:5]

array([806451572, 806451572, 806451572, 806455814, 806455814], dtype=int32)

## Explore tracks

In [84]:
r.events.tracks

<Branch[tracks]: 10 elements>

In [85]:
r.events.tracks._keymap.keys()

dict_keys(['rec_type', 'dir_z', 'pos_x', 'E', 'rec_stages', 'mother_id', 'pos_y', 'comment', 'lik', 't', 'hit_ids', 'type', 'fitinf', 'pos_z', 'error_matrix', 'usr_names', 'len', 'dir_y', 'dir_x', 'id', 'status'])

In [86]:
for key in r.events.tracks._keymap.keys():
    print(key, getattr(r.events.tracks, key), "\n")

rec_type [[4000 4000 4000 ... 4000 4000 4000] [4000 4000 4000 ... 4000 4000 4000] [4000 4000 4000 ... 4000 4000 4000] ... [4000 4000 4000 ... 4000 4000 4000] [4000 4000 4000 ... 4000 4000 4000] [4000 4000 4000 ... 4000 4000 4000]] 

dir_z [[-0.872885221293917 -0.872885221293917 -0.872885221293917 ... -0.6631226836266504 -0.5680647731737454 -0.5680647731737454] [-0.8351996698137462 -0.8351996698137462 -0.8351996698137462 ... -0.7485107718446855 -0.8229838871876581 -0.239315690284641] [-0.989148723802379 -0.989148723802379 -0.989148723802379 ... -0.9350162572437829 -0.88545604390297 -0.88545604390297] ... [-0.5704611045902105 -0.5704611045902105 -0.5704611045902105 ... -0.9350162572437829 -0.4647231989130516 -0.4647231989130516] [-0.9779941383490359 -0.9779941383490359 -0.9779941383490359 ... -0.88545604390297 -0.88545604390297 -0.8229838871876581] [-0.7396916780974963 -0.7396916780974963 -0.7396916780974963 ... -0.6631226836266504 -0.7485107718446855 -0.7485107718446855]] 

pos_x [[445.

In [87]:
r.events.tracks.lik

<ChunkedArray [[294.6407542676734 294.6407542676734 294.6407542676734 ... 67.81221253265059 67.7756405143316 67.77250505700384] [96.75133289411137 96.75133289411137 96.75133289411137 ... 39.21916536442286 39.184645826013806 38.870325146341884] [560.2775306614813 560.2775306614813 560.2775306614813 ... 118.88577278801066 118.72271313687405 117.80785995187605] ... [71.03251451148226 71.03251451148226 71.03251451148226 ... 16.714140573909347 16.444395245214945 16.34639241716669] [326.440133294878 326.440133294878 326.440133294878 ... 87.79818671079849 87.75488082571873 87.74839444768625] [159.77779654216795 159.77779654216795 159.77779654216795 ... 33.8669134999348 33.821631538334984 33.77240735670646]] at 0x7f74897e3d50>

## test tracks slicing

In [147]:
# this selects the first pos_y in all tracks in [3:6]
r.events.tracks.pos_y[3:6, 0]

<ChunkedArray [590.4753779242533 546.3344243172004 585.1168235271093] at 0x7f748984c950>

In [148]:
r.events[3:6].tracks.pos_y[:, 0]

<ChunkedArray [590.4753779242533 546.3344243172004 585.1168235271093] at 0x7f7489db0210>

In [149]:
r.events.tracks.pos_y[3][0]

590.4753779242533

In [150]:
r.events.tracks[0].lik[0:3]

array([294.64075427, 294.64075427, 294.64075427])

In [151]:
r.events[0].tracks.lik[0:3]

array([294.64075427, 294.64075427, 294.64075427])

## BUGS IN TRACKS SLICING

In [167]:
s = slice(0, 10, 3)
sliced_tracks = r.events.tracks[s]

In [161]:
sliced_tracks[0].lik[0:3]

array([294.64075427, 294.64075427, 294.64075427])

In [162]:
r.events.tracks[0].lik[0:3]

array([294.64075427, 294.64075427, 294.64075427])

In [163]:
sliced_tracks[1].lik[0:3]

array([96.75133289, 96.75133289, 96.75133289])

In [164]:
r.events.tracks[3].lik[0:3]

array([278.28729517, 278.28729517, 278.28729517])

In [165]:
sliced_tracks[2].lik[0:3]

array([560.27753066, 560.27753066, 560.27753066])

In [166]:
r.events.tracks[6].lik[0:3]

array([47.34583307, 47.34583307, 47.34583307])

## design unit tests for tracks

In [102]:
ki.offline._nested_mapper("tkrs.pos.x")

'pos_x'

In [103]:
class cached_property:
    """A simple cache decorator for properties."""
    def __init__(self, function):
        self.function = function

    def __get__(self, obj, cls):
        print(obj, cls)
        if obj is None:
            print("in None case")
            return self
        prop = obj.__dict__[self.function.__name__] = self.function(obj)
        return prop

In [104]:
class test:    
    @cached_property
    def lolo(self):
        print("calling lolo")
        return "with zizi"

In [108]:
t = test()

In [109]:
test.lolo

None <class '__main__.test'>
in None case


In [110]:
isinstance(test.lolo, cached_property)

None <class '__main__.test'>
in None case


True

In [111]:
t.__dict__

{}

In [113]:
ki.offline._to_num('test')

'test'

In [114]:
def _to_num(value):
    try:
        return int(value)
    except (ValueError, TypeError):
        pass
    try:
        return float(value)
    except (ValueError, TypeError):
        pass
    return value

In [115]:
type(_to_num(None))

NoneType

In [116]:
_to_num("10")

10

In [117]:
_to_num("10.5")

10.5

In [118]:
_to_num("test")

'test'

In [119]:
print(r3.header)

MC Header:
  DAQ(livetime=394)
  PDF(i1=4, i2=58)
  can(zmin=0, zmax=1027, r=888.4)
  can_user: can_user(field_0=0.0, field_1=1027.0, field_2=888.4)
  coord_origin(x=0, y=0, z=0)
  cut_in(Emin=0, Emax=0, cosTmin=0, cosTmax=0)
  cut_nu(Emin=100, Emax=100000000.0, cosTmin=-1, cosTmax=1)
  cut_primary(Emin=0, Emax=0, cosTmin=0, cosTmax=0)
  cut_seamuon(Emin=0, Emax=0, cosTmin=0, cosTmax=0)
  decay: decay(field_0='doesnt', field_1='happen')
  detector: NOT
  drawing: Volume
  genhencut(gDir=2000, Emin=0)
  genvol(zmin=0, zmax=1027, r=888.4, volume=2649000000.0, numberOfEvents=100000)
  kcut: 2
  livetime(numberOfSeconds=0, errorOfSeconds=0)
  model(interaction=1, muon=2, scattering=0, numberOfEnergyBins=1, field_4=12)
  ngen: 100000.0
  norma(primaryFlux=0, numberOfPrimaries=0)
  nuflux: nuflux(field_0=0, field_1=3, field_2=0, field_3=0.5, field_4=0.0, field_5=1.0, field_6=3.0)
  physics(program='GENHEN', version='7.2-220514', date=181116, time=1138)
  seed(program='GENHEN', level=3, iseed

In [121]:
from km3io.offline import Header

In [132]:
from km3io.definitions import mc_header

In [133]:
mc_header

{'DAQ': ['livetime'],
 'seed': ['program', 'level', 'iseed', 'field_3', 'field_4'],
 'PM1_type_area': ['type', 'area', 'TTS'],
 'PDF': ['i1', 'i2'],
 'model': ['interaction',
  'muon',
  'scattering',
  'numberOfEnergyBins',
  'field_4'],
 'can': ['zmin', 'zmax', 'r'],
 'genvol': ['zmin', 'zmax', 'r', 'volume', 'numberOfEvents'],
 'merge': ['time', 'gain'],
 'coord_origin': ['x', 'y', 'z'],
 'translate': ['x', 'y', 'z'],
 'genhencut': ['gDir', 'Emin'],
 'k40': ['rate', 'time'],
 'norma': ['primaryFlux', 'numberOfPrimaries'],
 'livetime': ['numberOfSeconds', 'errorOfSeconds'],
 'flux': ['type', 'key', 'file_1', 'file_2'],
 'spectrum': ['alpha'],
 'fixedcan': ['xcenter', 'ycenter', 'zmin', 'zmax', 'radius'],
 'start_run': ['run_id'],
 'cut_primary': ['Emin', 'Emax', 'cosTmin', 'cosTmax'],
 'cut_seamuon': ['Emin', 'Emax', 'cosTmin', 'cosTmax'],
 'cut_in': ['Emin', 'Emax', 'cosTmin', 'cosTmax'],
 'cut_nu': ['Emin', 'Emax', 'cosTmin', 'cosTmax'],
 'generator': ['program', 'version', 'date',

In [124]:
r3.header._data

{'DAQ': DAQ(livetime=394),
 'PDF': PDF(i1=4, i2=58),
 'can': can(zmin=0, zmax=1027, r=888.4),
 'can_user': can_user(field_0=0.0, field_1=1027.0, field_2=888.4),
 'coord_origin': coord_origin(x=0, y=0, z=0),
 'cut_in': cut_in(Emin=0, Emax=0, cosTmin=0, cosTmax=0),
 'cut_nu': cut_nu(Emin=100, Emax=100000000.0, cosTmin=-1, cosTmax=1),
 'cut_primary': cut_primary(Emin=0, Emax=0, cosTmin=0, cosTmax=0),
 'cut_seamuon': cut_seamuon(Emin=0, Emax=0, cosTmin=0, cosTmax=0),
 'decay': decay(field_0='doesnt', field_1='happen'),
 'detector': 'NOT',
 'drawing': 'Volume',
 'genhencut': genhencut(gDir=2000, Emin=0),
 'genvol': genvol(zmin=0, zmax=1027, r=888.4, volume=2649000000.0, numberOfEvents=100000),
 'kcut': 2,
 'livetime': livetime(numberOfSeconds=0, errorOfSeconds=0),
 'model': model(interaction=1, muon=2, scattering=0, numberOfEnergyBins=1, field_4=12),
 'ngen': 100000.0,
 'norma': norma(primaryFlux=0, numberOfPrimaries=0),
 'nuflux': nuflux(field_0=0, field_1=3, field_2=0, field_3=0.5, field_

In [125]:
head2 = {'DAQ': '394',
         'PDF': '4      58',
         'XSecFile': '',
         'can': '0 1027 888.4'}

fields = {'DAQ': ['livetime'],
          'seed': ['program', 'level', 'iseed'],
          'PM1_type_area': ['type', 'area', 'TTS'],
          'PDF': ['i1', 'i2']}

h2 = ki.offline.Header(head2)

In [126]:
h2._data

{'DAQ': DAQ(livetime=394),
 'PDF': PDF(i1=4, i2=58),
 'can': can(zmin=0, zmax=1027, r=888.4)}

In [127]:
h2.PDF

PDF(i1=4, i2=58)

In [128]:
long_h = {'DAQ': '394',
 'PDF': '4      58',
 'XSecFile': '',
 'can': '0 1027 888.4',
 'can_user': '0.00 1027.00  888.40',
 'coord_origin': '0 0 0',
 'cut_in': '0 0 0 0',
 'cut_nu': '100 1e+08 -1 1',
 'cut_primary': '0 0 0 0',
 'cut_seamuon': '0 0 0 0',
 'decay': 'doesnt happen',
 'detector': 'NOT',
 'drawing': 'Volume',
 'end_event': '',
 'genhencut': '2000 0',
 'genvol': '0 1027 888.4 2.649e+09 100000',
 'kcut': '2',
 'livetime': '0 0',
 'model': '1       2       0       1      12',
 'muon_desc_file': '',
 'ngen': '0.1000E+06',
 'norma': '0 0',
 'nuflux': '0       3       0 0.500E+00 0.000E+00 0.100E+01 0.300E+01',
 'physics': 'GENHEN 7.2-220514 181116 1138',
 'seed': 'GENHEN 3  305765867         0         0',
 'simul': 'JSirene 11012 11/17/18 07',
 'sourcemode': 'diffuse',
 'spectrum': '-1.4',
 'start_run': '1',
 'target': 'isoscalar',
 'usedetfile': 'false',
 'xlat_user': '0.63297',
 'xparam': 'OFF',
 'zed_user': '0.00 3450.00'}

In [129]:
long_head = ki.offline.Header(long_h)
len(long_head._data)

31

In [130]:
print(long_head)

MC Header:
  DAQ(livetime=394)
  PDF(i1=4, i2=58)
  can(zmin=0, zmax=1027, r=888.4)
  can_user: can_user(field_0=0.0, field_1=1027.0, field_2=888.4)
  coord_origin(x=0, y=0, z=0)
  cut_in(Emin=0, Emax=0, cosTmin=0, cosTmax=0)
  cut_nu(Emin=100, Emax=100000000.0, cosTmin=-1, cosTmax=1)
  cut_primary(Emin=0, Emax=0, cosTmin=0, cosTmax=0)
  cut_seamuon(Emin=0, Emax=0, cosTmin=0, cosTmax=0)
  decay: decay(field_0='doesnt', field_1='happen')
  detector: NOT
  drawing: Volume
  genhencut(gDir=2000, Emin=0)
  genvol(zmin=0, zmax=1027, r=888.4, volume=2649000000.0, numberOfEvents=100000)
  kcut: 2
  livetime(numberOfSeconds=0, errorOfSeconds=0)
  model(interaction=1, muon=2, scattering=0, numberOfEnergyBins=1, field_4=12)
  ngen: 100000.0
  norma(primaryFlux=0, numberOfPrimaries=0)
  nuflux: nuflux(field_0=0, field_1=3, field_2=0, field_3=0.5, field_4=0.0, field_5=1.0, field_6=3.0)
  physics(program='GENHEN', version='7.2-220514', date=181116, time=1138)
  seed(program='GENHEN', level=3, iseed

In [134]:
set(mc_header.keys())

{'DAQ',
 'PDF',
 'PM1_type_area',
 'can',
 'coord_origin',
 'cut_in',
 'cut_nu',
 'cut_primary',
 'cut_seamuon',
 'fixedcan',
 'flux',
 'generator',
 'genhencut',
 'genvol',
 'k40',
 'livetime',
 'merge',
 'model',
 'norma',
 'physics',
 'seed',
 'simul',
 'spectrum',
 'start_run',
 'translate'}